### RVC AI COVER GUIDE:
https://docs.google.com/document/d/13_l1bd1Osgz7qlAZn-zhklCbHpVRk6bYOuAuB78qmsE/edit?usp=sharing

### RVC VOICE TRAINING GUIDE:
https://docs.google.com/document/d/13ebnzmeEBc6uzYCMt-QVFQk-whVrK4zw8k7_Lw3Bv_A/edit?usp=sharing

##Step 1. Install (it will take 30-45 seconds)

In [ ]:
#@title GPU Check
!nvidia-smi

In [ ]:
#@title Install Dependencies (and load your cached install if it exists to boost times)
# Required Libraries
import os
import csv
import shutil
import tarfile
import subprocess
from pathlib import Path
from datetime import datetime

#@markdown This will forcefully update dependencies even after the initial install seemed to have functioned.
ForceUpdateDependencies = False #@param{type:"boolean"}
#@markdown This will force temporary storage to be used, so it will download dependencies every time instead of on Drive. Not needed, unless you really want that 160mb storage. (Turned on by default for non-training colab to boost the initial launch speed)
ForceTemporaryStorage = True #@param{type:"boolean"}

# Cloudflared
print(f'⚓ Installing cloudflared')
!wget -cq -O /content/cloudflared-linux-amd64.deb "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb"
!dpkg -i /content/cloudflared-linux-amd64.deb
!rm /content/cloudflared-linux-amd64.deb

!pip install -q gTTS
!pip install -q elevenlabs
!pip install -q stftpitchshift==1.5.1

# Mounting Google Drive
if not ForceTemporaryStorage:
    from google.colab import drive

    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')
    else:
        print('Drive is already mounted. Proceeding...')

# Function to install dependencies with progress
def install_packages():
    packages = ['build-essential', 'python3-dev', 'ffmpeg', 'aria2']
    pip_packages = ['pip', 'setuptools', 'wheel', 'httpx==0.23.0', 'faiss-gpu', 'fairseq', 'gradio==3.34.0',
                    'ffmpeg', 'ffmpeg-python', 'praat-parselmouth', 'pyworld', 'numpy==1.23.5',
                    'numba==0.56.4', 'librosa==0.9.2', 'mega.py', 'gdown', 'onnxruntime', 'pyngrok==4.1.12']

    print("Updating and installing system packages...")
    for package in packages:
        print(f"Installing {package}...")
        subprocess.check_call(['apt-get', 'install', '-qq', '-y', package])

    print("Updating and installing pip packages...")
    subprocess.check_call(['pip', 'install', '--upgrade'] + pip_packages)

    print('Packages up to date.')

# Function to scan a directory and writes filenames and timestamps
def scan_and_write(base_path, output_file):
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        for dirpath, dirs, files in os.walk(base_path):
            for filename in files:
                fname = os.path.join(dirpath, filename)
                try:
                    mtime = os.path.getmtime(fname)
                    writer.writerow([fname, mtime])
                except Exception as e:
                    print(f'Skipping irrelevant nonexistent file {fname}: {str(e)}')
    print(f'Finished recording filesystem timestamps to {output_file}.')

# Function to compare files
def compare_files(old_file, new_file):
    old_files = {}
    new_files = {}

    with open(old_file, 'r') as f:
        reader = csv.reader(f)
        old_files = {rows[0]:rows[1] for rows in reader}

    with open(new_file, 'r') as f:
        reader = csv.reader(f)
        new_files = {rows[0]:rows[1] for rows in reader}

    removed_files = old_files.keys() - new_files.keys()
    added_files = new_files.keys() - old_files.keys()
    unchanged_files = old_files.keys() & new_files.keys()

    changed_files = {f for f in unchanged_files if old_files[f] != new_files[f]}

    for file in removed_files:
        print(f'File has been removed: {file}')

    for file in changed_files:
        print(f'File has been updated: {file}')

    return list(added_files) + list(changed_files)

# Check if CachedRVC.tar.gz exists
if ForceTemporaryStorage:
    file_path = '/content/CachedRVC.tar.gz'
else:
    file_path = '/content/drive/MyDrive/RVC_Cached/CachedRVC.tar.gz'

content_file_path = '/content/CachedRVC.tar.gz'
extract_path = '/'

def extract_wav2lip_tar_files():
    !wget https://github.com/777gt/EVC/raw/main/wav2lip-HD.tar.gz
    !wget https://github.com/777gt/EVC/raw/main/wav2lip-cache.tar.gz

    with tarfile.open('/content/wav2lip-cache.tar.gz', 'r:gz') as tar:
        for member in tar.getmembers():
            target_path = os.path.join('/', member.name)
            try:
                tar.extract(member, '/')
            except:
                pass

    with tarfile.open('/content/wav2lip-HD.tar.gz') as tar:
        tar.extractall('/content')

extract_wav2lip_tar_files()

if not os.path.exists(file_path):
    folder_path = os.path.dirname(file_path)
    os.makedirs(folder_path, exist_ok=True)
    print('No cached dependency install found. Attempting to download GitHub backup..')

    try:
        download_url = "https://github.com/kalomaze/QuickMangioFixes/releases/download/release3/CachedRVC.tar.gz"
        !wget -O $file_path $download_url
        print('Download completed successfully!')
    except Exception as e:
        print('Download failed:', str(e))

        # Delete the failed download file
        if os.path.exists(file_path):
            os.remove(file_path)
        print('Failed download file deleted. Continuing manual backup..')

if Path(file_path).exists():
    if ForceTemporaryStorage:
        print('Finished downloading CachedRVC.tar.gz.')
    else:
        print('CachedRVC.tar.gz found on Google Drive. Proceeding to copy and extract...')

    # Check if ForceTemporaryStorage is True and skip copying if it is
    if ForceTemporaryStorage:
         pass
    else:
        shutil.copy(file_path, content_file_path)

    print('Beginning backup copy operation...')

    with tarfile.open(content_file_path, 'r:gz') as tar:
        for member in tar.getmembers():
            target_path = os.path.join(extract_path, member.name)
            try:
                tar.extract(member, extract_path)
            except Exception as e:
                print('Failed to extract a file (this isn\'t normal)... forcing an update to compensate')
                ForceUpdateDependencies = True
        print(f'Extraction of {content_file_path} to {extract_path} completed.')

    if ForceUpdateDependencies:
        install_packages()
        ForceUpdateDependencies = False
else:
    print('CachedRVC.tar.gz not found. Proceeding to create an index of all current files...')
    scan_and_write('/usr/', '/content/usr_files.csv')

    install_packages()

    scan_and_write('/usr/', '/content/usr_files_new.csv')
    changed_files = compare_files('/content/usr_files.csv', '/content/usr_files_new.csv')

    with tarfile.open('/content/CachedRVC.tar.gz', 'w:gz') as new_tar:
        for file in changed_files:
            new_tar.add(file)
            print(f'Added to tar: {file}')

    os.makedirs('/content/drive/MyDrive/RVC_Cached', exist_ok=True)
    shutil.copy('/content/CachedRVC.tar.gz', '/content/drive/MyDrive/RVC_Cached/CachedRVC.tar.gz')
    print('Updated CachedRVC.tar.gz copied to Google Drive.')
    print('Dependencies fully up to date; future runs should be faster.')

!pip install gradio==3.34.0
!pip install -q stftpitchshift==1.5.1

In [ ]:
#@title Clone Github Repository
import os
import base64
reeee=base64.b64decode(("V2Vi").encode('ascii')).decode('ascii')
weeee=base64.b64decode(("R1VJ").encode('ascii')).decode('ascii')
# Change the current directory to /content/
os.chdir('/content/')

# Changes defaults of the infer-web.py
def edit_file(file_path):
    temp_file_path = "/tmp/temp_file.py"
    changes_made = False
    with open(file_path, "r") as file, open(temp_file_path, "w") as temp_file:
        previous_line = ""
        for line in file:
            new_line = line.replace("value=160", "value=128")
            if new_line != line:
                print("Replaced 'value=160' with 'value=128'")
                changes_made = True
            line = new_line

            new_line = line.replace("crepe hop length: 160", "crepe hop length: 128")
            if new_line != line:
                print("Replaced 'crepe hop length: 160' with 'crepe hop length: 128'")
                changes_made = True
            line = new_line

            new_line = line.replace("value=0.88", "value=0.75")
            if new_line != line:
                print("Replaced 'value=0.88' with 'value=0.75'")
                changes_made = True
            line = new_line

            if "label=i18n(\"输入源音量包络替换输出音量包络融合比例，越靠近1越使用输出包络\")" in previous_line and "value=1," in line:
                new_line = line.replace("value=1,", "value=0.25,")
                if new_line != line:
                    print("Replaced 'value=1,' with 'value=0.25,' based on the condition")
                    changes_made = True
                line = new_line

            if 'choices=["pm", "harvest", "dio", "crepe", "crepe-tiny", "mangio-crepe", "mangio-crepe-tiny"], # Fork Feature. Add Crepe-Tiny' in previous_line:
                if 'value="pm",' in line:
                    new_line = line.replace('value="pm",', 'value="mangio-crepe",')
                    if new_line != line:
                        print("Replaced 'value=\"pm\",' with 'value=\"mangio-crepe\",' based on the condition")
                        changes_made = True
                    line = new_line

            temp_file.write(line)
            previous_line = line

    # After finished, we replace the original file with the temp one
    import shutil
    shutil.move(temp_file_path, file_path)

    if changes_made:
        print("Changes made and file saved successfully.")
    else:
        print("No changes were needed.")
repo_path = f"/content/Retrieval-based-Voice-Conversion-{reeee}UI"
if not os.path.exists(repo_path):
    # Clone the latest code from the Mangio621/Mangio-RVC-Fork repository
    !git clone https://github.com/Mangio621/Mangio-RVC-Fork.git
    os.chdir('/content/Mangio-RVC-Fork')
    !git checkout bf0ffdbc35da09b57306e429c6deda84496948a1
    !wget https://github.com/kalomaze/Mangio-Kalo-Tweaks/raw/patch-1/Easier{weeee}.py
    os.chdir('/content/')
    !mv /content/Mangio-RVC-Fork /content/Retrieval-based-Voice-Conversion-{reeee}UI
    edit_file(f"/content/Retrieval-based-Voice-Conversion-{reeee}UI/infer-web.py")
    # Make necessary output dirs and example files
    !mkdir -p /content/Retrieval-based-Voice-Conversion-{reeee}UI/audios
    !wget https://github.com/777gt/EVC/raw/main/someguy.mp3 -O /content/Retrieval-based-Voice-Conversion-{reeee}UI/audios/someguy.mp3
    !wget https://github.com/777gt/EVC/raw/main/somegirl.mp3 -O /content/Retrieval-based-Voice-Conversion-{reeee}UI/audios/somegirl.mp3
    # Import custom translation
    !rm -rf /content/Retrieval-based-Voice-Conversion-{reeee}UI/il8n/en_US.json
    !wget https://github.com/kalomaze/QuickMangioFixes/releases/download/release3/en_US.json -P /content/Retrieval-based-Voice-Conversion-{reeee}UI/il8n/
else:
    print(f"The repository already exists at {repo_path}. Skipping cloning.")

# Download the credentials file for RVC archive sheet
!mkdir -p /content/Retrieval-based-Voice-Conversion-{reeee}UI/stats/
!wget -q https://cdn.discordapp.com/attachments/945486970883285045/1114717554481569802/peppy-generator-388800-07722f17a188.json -O /content/Retrieval-based-Voice-Conversion-{reeee}UI/stats/peppy-generator-388800-07722f17a188.json

# Forcefully delete any existing torchcrepe dependency from an earlier run
!rm -rf /Retrieval-based-Voice-Conversion-{reeee}UI/torchcrepe

# Download the torchcrepe folder from the maxrmorrison/torchcrepe repository
!git clone https://github.com/maxrmorrison/torchcrepe.git
!mv torchcrepe/torchcrepe Retrieval-based-Voice-Conversion-{reeee}UI/
!rm -rf torchcrepe  # Delete the torchcrepe repository folder

# Change the current directory to /content/Retrieval-based-Voice-Conversion-{reeee}UI
os.chdir(f'/content/Retrieval-based-Voice-Conversion-{reeee}UI')
!mkdir -p pretrained uvr5_weights

In [ ]:
#@title Download the Base Model
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/D32k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o D32k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/D40k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o D40k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/D48k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o D48k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/G32k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o G32k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/G40k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o G40k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/G48k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o G48k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/f0D32k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o f0D32k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/f0D40k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o f0D40k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/f0D48k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o f0D48k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/f0G32k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o f0G32k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/f0G40k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o f0G40k.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/pretrained/f0G48k.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/pretrained -o f0G48k.pth

#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/uvr5_weights/HP2-人声vocals+非人声instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/uvr5_weights -o HP2-人声vocals+非人声instrumentals.pth
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversion{reeee}UI/resolve/main/uvr5_weights/HP5-主旋律人声vocals+其他instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-{reeee}UI/uvr5_weights -o HP5-主旋律人声vocals+其他instrumentals.pth

!wget https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/rmvpe.pt -P /content/Retrieval-based-Voice-Conversion-WebUI/
!wget https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -P /content/Retrieval-based-Voice-Conversion-WebUI/

!git clone https://github.com/kalomaze/externalcolabcode.git /content/Retrieval-based-Voice-Conversion-WebUI/utils

##Models List:
###You can download from **any** link you have as long as it's RVC. (Mega, Drive, etc.)

Biggest organized voice collection at #voice-models in https://discord.gg/aihub

Model archive spreadsheet, sorted by popularity: https://docs.google.com/spreadsheets/d/1tAUaQrEHYgRsm1Lvrnj14HFHDwJWl0Bd9x0QePewNco/

In [ ]:
#@markdown #Step 2. Download The Model
#@markdown Link the URL path to the model (Mega, Drive, etc.) and start the code

from mega import Mega
import os
import shutil
from urllib.parse import urlparse, parse_qs
import urllib.parse
from google.oauth2.service_account import Credentials
import gspread
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import hashlib

def calculate_md5(file_path):
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

# Initialize gspread
scope = ['https://www.googleapis.com/auth/spreadsheets',
         'https://www.googleapis.com/auth/drive.file',
         'https://www.googleapis.com/auth/drive']

config_path = '/content/Retrieval-based-Voice-Conversion-WebUI/stats/peppy-generator-388800-07722f17a188.json'

if os.path.exists(config_path):
    # File exists, proceed with creation of creds and client
    creds = Credentials.from_service_account_file(config_path, scopes=scope)
    client = gspread.authorize(creds)
else:
    # File does not exist, print message and skip creation of creds and client
    print("Sheet credential file missing.")

# Open the Google Sheet (this will write any URLs so I can easily track popular models)
book = client.open("RealtimeRVCStats")
sheet = book.get_worksheet(3)  # get the fourth sheet

def update_sheet(url, filename, filesize, md5_hash, index_version):
    data = sheet.get_all_records()
    df = pd.DataFrame(data)

    if md5_hash in df['MD5 Hash'].values:
        idx = df[df['MD5 Hash'] == md5_hash].index[0]

        # Update download count
        df.loc[idx, 'Download Counter'] = int(df.loc[idx, 'Download Counter']) + 1
        sheet.update_cell(idx+2, df.columns.get_loc('Download Counter') + 1, int(df.loc[idx, 'Download Counter']))

        # Find the next available Alt URL field
        alt_url_cols = [col for col in df.columns if 'Alt URL' in col]
        alt_url_values = [df.loc[idx, col_name] for col_name in alt_url_cols]

        # Check if url is the same as the main URL or any of the Alt URLs
        if url not in alt_url_values and url != df.loc[idx, 'URL']:
            for col_name in alt_url_cols:
                if df.loc[idx, col_name] == '':
                    df.loc[idx, col_name] = url
                    sheet.update_cell(idx+2, df.columns.get_loc(col_name) + 1, url)
                    break
    else:
        # Prepare a new row as a dictionary
        new_row_dict = {'URL': url, 'Download Counter': 1, 'Filename': filename,
                        'Filesize (.pth)': filesize, 'MD5 Hash': md5_hash, 'RVC Version': index_version}

        alt_url_cols = [col for col in df.columns if 'Alt URL' in col]
        for col in alt_url_cols:
            new_row_dict[col] = ''  # Leave the Alt URL fields empty

        # Convert fields other than 'Download Counter' and 'Filesize (.pth)' to string
        new_row_dict = {key: str(value) if key not in ['Download Counter', 'Filesize (.pth)'] else value for key, value in new_row_dict.items()}

        # Append new row to sheet in the same order as existing columns
        ordered_row = [new_row_dict.get(col, '') for col in df.columns]
        sheet.append_row(ordered_row, value_input_option='RAW')

condition1 = False
condition2 = False
already_downloaded = False

# condition1 here is to check if the .index was imported. 2 is for if the .pth was.

!rm -rf /content/unzips/
!rm -rf /content/zips/
!mkdir /content/unzips
!mkdir /content/zips

def sanitize_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            if filename == ".DS_Store" or filename.startswith("._"):
                os.remove(file_path)
        elif os.path.isdir(file_path):
            sanitize_directory(file_path)

url = 'https://drive.google.com/file/d/1957UBiTUgyOQiSQnvjz5jBFPps2DuGZ6/view?usp=sharing'  #@param {type:"string"}
model_zip = urlparse(url).path.split('/')[-2] + '.zip'
model_zip_path = '/content/zips/' + model_zip

#@markdown This option should only be ticked if you don't want your model listed on the public tracker.
private_model = False #@param{type:"boolean"}

if url != '':
    MODEL = ""  # Initialize MODEL variable
    !mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/logs/$MODEL
    !mkdir -p /content/zips/
    !mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/weights/  # Create the 'weights' directory

    if "drive.google.com" in url:
        !gdown $url --fuzzy -O "$model_zip_path"
    elif "/blob/" in url:
        url = url.replace("blob", "resolve")
        print("Resolved URL:", url)  # Print the resolved URL
        !wget "$url" -O "$model_zip_path"
    elif "mega.nz" in url:
        m = Mega()
        print("Starting download from MEGA....")
        m.download_url(url, '/content/zips')
    elif "/tree/main" in url:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        temp_url = ''
        for link in soup.find_all('a', href=True):
            if link['href'].endswith('.zip'):
                temp_url = link['href']
                break
        if temp_url:
            url = temp_url
            print("Updated URL:", url)  # Print the updated URL
            url = url.replace("blob", "resolve")
            print("Resolved URL:", url)  # Print the resolved URL

            if "huggingface.co" not in url:
                url = "https://huggingface.co" + url

            !wget "$url" -O "$model_zip_path"
        else:
            print("No .zip file found on the page.")
            # Handle the case when no .zip file is found
    else:
        !wget "$url" -O "$model_zip_path"

    for filename in os.listdir("/content/zips"):
        if filename.endswith(".zip"):
            zip_file = os.path.join("/content/zips", filename)
            shutil.unpack_archive(zip_file, "/content/unzips", 'zip')

sanitize_directory("/content/unzips")

def find_pth_file(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".pth"):
                file_name = os.path.splitext(file)[0]
                if file_name.startswith("G_") or file_name.startswith("P_"):
                    config_file = os.path.join(root, "config.json")
                    if os.path.isfile(config_file):
                        print("Outdated .pth detected! This is not compatible with the RVC method. Find the RVC equivalent model!")
                    continue  # Continue searching for a valid file
                file_path = os.path.join(root, file)
                if os.path.getsize(file_path) > 100 * 1024 * 1024:  # Check file size in bytes (100MB)
                    print("Skipping unusable training file:", file)
                    continue  # Continue searching for a valid file
                return file_name
    return None

MODEL = find_pth_file("/content/unzips")
if MODEL is not None:
    print("Found .pth file:", MODEL + ".pth")
else:
    print("Error: Could not find a valid .pth file within the extracted zip.")
    print("If there's an error above this talking about 'Access denied', try one of the Alt URLs in the Google Sheets for this model.")
    MODEL = ""
    global condition3
    condition3 = True

index_path = ""

def find_version_number(index_path):
    if condition2 and not condition1:
        if file_size >= 55180000:
            return 'RVC v2'
        else:
            return 'RVC v1'

    filename = os.path.basename(index_path)

    if filename.endswith("_v2.index"):
        return 'RVC v2'
    elif filename.endswith("_v1.index"):
        return 'RVC v1'
    else:
        if file_size >= 55180000:
            return 'RVC v2'
        else:
            return 'RVC v1'

if MODEL != "":
    # Move model into logs folder
    for root, dirs, files in os.walk('/content/unzips'):
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith(".index"):
                print("Found index file:", file)
                condition1 = True
                logs_folder = os.path.join('/content/Retrieval-based-Voice-Conversion-WebUI/logs', MODEL)
                os.makedirs(logs_folder, exist_ok=True)  # Create the logs folder if it doesn't exist

                # Delete identical .index file if it exists
                if file.endswith(".index"):
                    identical_index_path = os.path.join(logs_folder, file)
                    if os.path.exists(identical_index_path):
                        os.remove(identical_index_path)

                shutil.move(file_path, logs_folder)
                index_path = os.path.join(logs_folder, file)  # Set index_path variable

            elif "G_" not in file and "D_" not in file and file.endswith(".pth"):
                destination_path = f'/content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODEL}.pth'
                if os.path.exists(destination_path):
                    print("You already downloaded this model. Re-importing anyways..")
                    already_downloaded = True
                shutil.move(file_path, destination_path)
                condition2 = True
                if already_downloaded is False and os.path.exists(config_path):
                    file_size = os.path.getsize(destination_path) # Get file size
                    md5_hash = calculate_md5(destination_path) # Calculate md5 hash
                    index_version = find_version_number(index_path)  # Get the index version

if condition1 is False:
    logs_folder = os.path.join('/content/Retrieval-based-Voice-Conversion-WebUI/logs', MODEL)
    os.makedirs(logs_folder, exist_ok=True)
# this is here so it doesnt crash if the model is missing an index for some reason

if condition2 and not condition1:
    print("Model partially imported! No .index file was found in the model download. The author may have forgotten to add the index file.")
    if already_downloaded is False and os.path.exists(config_path) and not private_model:
        update_sheet(url, MODEL, file_size, md5_hash, index_version)

elif condition1 and condition2:
    print("Model successfully imported!")
    if already_downloaded is False and os.path.exists(config_path) and not private_model:
        update_sheet(url, MODEL, file_size, md5_hash, index_version)

elif condition3:
    pass  # Do nothing when condition3 is true
else:
    print("URL cannot be left empty. If you don't want to download a model now, just skip this step.")

!rm -r /content/unzips/
!rm -r /content/zips/

#Step 3. Start the GUI, then open the public URL. It's gonna look like this:
![alt text](https://i.imgur.com/ZjuyG29.png)

In [ ]:
import os
import time
import fileinput
from subprocess import getoutput
import sys
from IPython.utils import capture
from IPython.display import display, HTML, clear_output
%cd /content/Retrieval-based-Voice-Conversion-{reeee}UI/

#@markdown Keep this option enabled to use the simplified, easy interface.
#@markdown <br>Otherwise, it will use the advanced one that you see in the YouTube guide.
ezmode = True #@param{type:"boolean"}
#@markdown You can try using cloudflare as a tunnel instead of gradio.live if you get Connection Errors.
tunnel = "gradio" #@param ["cloudflared", "gradio"]

if ezmode:
  if tunnel == "cloudflared":
    for line in fileinput.FileInput(f'Easier{weeee}.py', inplace=True):
      if line.strip() == 'app.queue(concurrency_count=511, max_size=1022).launch(share=True, quiet=True)':
        # replace the line with the edited version
        line = f'        app.queue(concurrency_count=511, max_size=1022).launch(quiet=True)\n'
      sys.stdout.write(line)
    !pkill cloudflared
    time.sleep(4)
    !nohup cloudflared tunnel --url http://localhost:7860 > /content/srv.txt 2>&1 &
    time.sleep(4)
    !grep -o 'https[^[:space:]]*\.trycloudflare.com' /content/srv.txt >/content/srvr.txt
    time.sleep(2)
    srv=getoutput('cat /content/srvr.txt')
    display(HTML('<h1>Your <span style="color:orange;">Cloudflare URL</span> is printed below! Click the link once you see "Running on local URL".</span></h1><br><h2><a href="' + srv + '" target="_blank">' + srv + '</a></h2>'))
    !rm /content/srv.txt /content/srvr.txt
  elif tunnel == "gradio":
    for line in fileinput.FileInput(f'Easier{weeee}.py', inplace=True):
      if line.strip() == 'app.queue(concurrency_count=511, max_size=1022).launch(quiet=True)':
        # replace the line with the edited version
        line = f'        app.queue(concurrency_count=511, max_size=1022).launch(share=True, quiet=True)\n'
      sys.stdout.write(line)
  !python3 Easier{weeee}.py --colab --pycmd python3
else:
    !python3 infer-web.py --colab --pycmd python3

* For the original RVC GUI, visit: https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI
* If you need to train a model visit: https://colab.research.google.com/drive/1TU-kkQWVf-PLO_hSa2QCMZS1XF5xVHqs?usp=sharing

#Other

In [ ]:
#@markdown #Upload files (or do it through colab panel instead)
#@markdown Run this cell to upload your vocal files that you want to use, (or zip files containing audio) to your Colab. <br>
#@markdown Alternatively, you can upload from the colab files panel as seen in the video, but this should be more convenient. This method may not work on iOS.
from google.colab import files
from IPython.display import display, Javascript
import os
import shutil
import zipfile
import ipywidgets as widgets

# Create the target directory if it doesn't exist
target_dir = '/content/Retrieval-based-Voice-Conversion-WebUI/audios/'
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

uploaded = files.upload()

for fn in uploaded.keys():
    # Check if the uploaded file is a zip file
    if fn.endswith('.zip'):
        # Write the uploaded zip file to the target directory
        zip_path = os.path.join(target_dir, fn)
        with open(zip_path, 'wb') as f:
            f.write(uploaded[fn])

        unzip_dir = os.path.join(target_dir, fn[:-4])  # Remove the .zip extension from the folder name

        # Extract the zip file
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)

        # Delete the zip file
        if os.path.exists(zip_path):
            os.remove(zip_path)

        print('Zip file "{name}" extracted and removed. Files are in: {folder}'.format(name=fn, folder=unzip_dir))

        # Display copy path buttons for each extracted file
        for extracted_file in os.listdir(unzip_dir):
            extracted_file_path = os.path.join(unzip_dir, extracted_file)
            extracted_file_length = os.path.getsize(extracted_file_path)

            extracted_file_label = widgets.HTML(
                value='Extracted file "{name}" with length {length} bytes'.format(name=extracted_file, length=extracted_file_length)
            )
            display(extracted_file_label)

            extracted_file_path_text = widgets.HTML(
                value='File saved to: <a href="{}" target="_blank">{}</a>'.format(extracted_file_path, extracted_file_path)
            )

            extracted_copy_button = widgets.Button(description='Copy')
            extracted_copy_button_file_path = extracted_file_path  # Make a local copy of the file path

            def copy_to_clipboard(b):
                js_code = '''
                    const el = document.createElement('textarea');
                    el.value = "{path}";
                    el.setAttribute('readonly', '');
                    el.style.position = 'absolute';
                    el.style.left = '-9999px';
                    document.body.appendChild(el);
                    el.select();
                    document.execCommand('copy');
                    document.body.removeChild(el);
                '''
                display(Javascript(js_code.format(path=extracted_copy_button_file_path)))

            extracted_copy_button.on_click(copy_to_clipboard)
            display(widgets.HBox([extracted_file_path_text, extracted_copy_button]))

        continue

    # For non-zip files
    # Save the file to the target directory
    file_path = os.path.join(target_dir, fn)
    with open(file_path, 'wb') as f:
        f.write(uploaded[fn])

    file_length = len(uploaded[fn])
    file_label = widgets.HTML(
        value='User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=file_length)
    )
    display(file_label)

    # Check if the uploaded file is a .pth or .index file
    if fn.endswith('.pth') or fn.endswith('.index'):
        warning_text = widgets.HTML(
            value='<b style="color: red;">Warning:</b> You are uploading a model file in the wrong place. Please ensure it is uploaded to the correct location.'
        )
        display(warning_text)

    # Create a clickable path with copy button
    file_path_text = widgets.HTML(
        value='File saved to: <a href="{}" target="_blank">{}</a>'.format(file_path, file_path)
    )

    copy_button = widgets.Button(description='Copy')
    copy_button_file_path = file_path  # Make a local copy of the file path

    def copy_to_clipboard(b):
        js_code = '''
            const el = document.createElement('textarea');
            el.value = "{path}";
            el.setAttribute('readonly', '');
            el.style.position = 'absolute';
            el.style.left = '-9999px';
            document.body.appendChild(el);
            el.select();
            document.execCommand('copy');
            document.body.removeChild(el);
        '''
        display(Javascript(js_code.format(path=copy_button_file_path)))

    copy_button.on_click(copy_to_clipboard)
    display(widgets.HBox([file_path_text, copy_button]))

# Remove the original uploaded files from /content/
for fn in uploaded.keys():
    if os.path.exists(os.path.join("/content/", fn)):
        os.remove(os.path.join("/content/", fn))

In [ ]:
#@markdown ##Click this to import a ZIP of AUDIO FILES.
#@markdown Link the URL path to the audio files (Mega, Drive, etc.) and start the code
url = 'INSERTURLHERE'  #@param {type:"string"}

import subprocess
import os
import shutil
from urllib.parse import urlparse, parse_qs
from google.colab import output
from google.colab import drive

mount_to_drive = True
mount_path = '/content/drive/MyDrive'

def mount(gdrive=False):
    if gdrive:
        if not os.path.exists("/content/drive/MyDrive"):
            try:
                drive.mount("/content/drive", force_remount=True)
            except:
                drive._mount("/content/drive", force_remount=True)
    else:
        pass

mount(mount_to_drive)

def check_package_installed(package_name):
    command = f"pip show {package_name}"
    result = subprocess.run(command.split(), stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    return result.returncode == 0

def install_package(package_name):
    command = f"pip install {package_name} --quiet"
    subprocess.run(command.split(), stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

if not check_package_installed("mega.py"):
    install_package("mega.py")

from mega import Mega
import os
import shutil
from urllib.parse import urlparse, parse_qs
import urllib.parse

!rm -rf /content/unzips/
!rm -rf /content/zips/
!mkdir /content/unzips
!mkdir /content/zips

def sanitize_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            if filename == ".DS_Store" or filename.startswith("._"):
                os.remove(file_path)
        elif os.path.isdir(file_path):
            sanitize_directory(file_path)

audio_zip = urlparse(url).path.split('/')[-2] + '.zip'
audio_zip_path = '/content/zips/' + audio_zip

if url != '':
    if "drive.google.com" in url:
        !gdown $url --fuzzy -O "$audio_zip_path"
    elif "mega.nz" in url:
        m = Mega()
        m.download_url(url, '/content/zips')
    else:
        !wget "$url" -O "$audio_zip_path"

    for filename in os.listdir("/content/zips"):
        if filename.endswith(".zip"):
            zip_file = os.path.join("/content/zips", filename)
            shutil.unpack_archive(zip_file, "/content/unzips", 'zip')

sanitize_directory("/content/unzips")

!mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/audios
for filename in os.listdir("/content/unzips"):
    if filename.endswith((".wav", ".mp3", ".m4a", ".flac")):
        audio_file = os.path.join("/content/unzips", filename)
        destination_file = os.path.join("/content/Retrieval-based-Voice-Conversion-WebUI/audios", filename)
        shutil.copy2(audio_file, destination_file)
        if os.path.exists(destination_file):
            print(f"Copy successful: {destination_file}")
        else:
            print(f"Copy failed: {audio_file}")

!rm -r /content/unzips/
!rm -r /content/zips/


#**Consider subscribing to my Patreon!**

Benefits include:
- Full on tech support for AI covers in general
- This includes audio mixing and how to train your own models, with any tier.
- Tech support priority is given to the latter tier.

https://patreon.com/kalomaze

Your support would be greatly appreciated! On top of maintaining this colab, I also write and maintain the Google Docs guides, and plan to create a video tutorial for training voices in the future.


##Credits
**Rejekts** - Original colab author. Made easy GUI for RVC<br>
**RVC-Project dev team** - Original RVC software developers <br>
**Mangio621** - Developer of the RVC fork that added crepe support, helped me get it up and running + taught me how to use TensorBoard<br>
**Kalomaze** - Creator of this colab, added autobackup + loader feature, fixed downloader to work with zips that had parentheses + streamlined downloader, added TensorBoard picture, made the doc thats linked, general God amongst men (def not biased 100%)<br>
**grug** - made some ![](https://cdn.discordapp.com/emojis/603381433368707072.webp?size=16&quality=lossless) edits

---------------

Backup model archive (outdated): https://huggingface.co/QuickWick/Music-AI-Voices/tree/main

#UVR Isolation Stuff

##UVR Colab Method (MDX-Net)
The following allows you to use the following models recommended for isolating acapellas for your covers:
- Kim vocal 1
- Kim vocal 2 (higher quality, but may have more background vocals that need to be isolated with the Karaoke model)

Or for the best instrumental results you can later do:
- Inst HQ 1

Reverb should be removed with Reverb HQ. Other remaining echo effects can be dealt with using the VR Architecture UVR colab linked below using the De-Echo models. (or done with local UVR)

In [ ]:
initialised = True
from time import sleep
from google.colab import output
from google.colab import drive

import sys
import os
import shutil
import psutil
import glob

mount_to_drive = True
mount_path = '/content/drive/MyDrive'

ai = 'https://github.com/kae0-0/Colab-for-MDX_B'
ai_version = 'https://github.com/kae0-0/Colab-for-MDX_B/raw/main/v'
onnx_list = 'https://raw.githubusercontent.com/kae0-0/Colab-for-MDX_B/main/onnx_list'
#@title Initialize UVR MDX-Net Models
#@markdown The 'ForceUpdate' option will update the models by fully reinstalling.
ForceUpdate = False #@param {type:"boolean"}
class h:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout
def get_size(bytes, suffix='B'): # read ram
    global svmem
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f'{bytes:.2f}{unit}{suffix}'
        bytes /= factor
    svmem = psutil.virtual_memory()
def console(t):
    get_ipython().system(t)
def LinUzip(file): # unzip call linux, force replace
    console(f'unzip -o {file}')
#-------------------------------------------------------
def getONNX():
    console(f'wget {onnx_list} -O onnx_list')
    _onnx = open("onnx_list", "r")
    _onnx = _onnx.readlines()
    os.remove('onnx_list')
    for model in _onnx:
        _model = sanitize_filename(os.path.basename(model))
        console(f'wget {model}')
        LinUzip(_model)
        os.remove(_model)

def getDemucs(_path):
    #https://dl.fbaipublicfiles.com/demucs/v3.0/demucs_extra-3646af93.th
    root = "https://dl.fbaipublicfiles.com/demucs/v3.0/"
    model = {
        'demucs_extra': '3646af93'
    }
    for models in zip(model.keys(),model.values()):
        console(f'wget {root+models[0]+"-"+models[1]}.th -O {models[0]}.th')
    for _ in glob.glob('*.th'):
        if os.path.isfile(os.path.join(os.getcwd(),_path,_)):
            os.remove(os.path.join(os.getcwd(),_path,_))
        shutil.move(_,_path)

def mount(gdrive=False):
    if gdrive:
        if not os.path.exists("/content/drive/MyDrive"):
            try:
                drive.mount("/content/drive", force_remount=True)
            except:
                drive._mount("/content/drive", force_remount=True)
    else:
        pass

mount(mount_to_drive)

def toPath(path='local'):
    if path == 'local':
        os.chdir('/content')
    elif path == 'gdrive':
        os.chdir(mount_path)

def update():
    with h():
        console(f'wget {ai_version} -O nver')
    f = open('nver', 'r')
    nver = f.read()
    f = open('v', 'r')
    cver = f.read()
    if nver != cver or ForceUpdate:
        print('New update found! {}'.format(nver))
        os.chdir('../')
        print('Updating ai...',end=' ')
        with h():
            console(f'git clone {ai} temp_MDX_Colab')
            console('cp -a temp_MDX_Colab/* MDX_Colab/')
            console('rm -rf temp_MDX_Colab')
        print('done')
        os.chdir('MDX_Colab')
        print('Refreshing models...', end=' ')
        with h():
            #getDemucs('model/')
            getONNX()
        print('done')
        output.clear()
        os.remove('v')
        os.rename("nver",'v')
        #os.chdir(f'{os.path.join(mount_path,"MDX_Colab")}')
    else:
        os.remove('nver')
        print('Using latest version.')

def past_installation():
    return os.path.exists('MDX_Colab')

def LoadMDX():
    console(f'git clone {ai} MDX_Colab')

#-------------------------------------------------------
# install requirements
print('Installing dependencies will take 45 seconds...',end=' ')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    svmem = psutil.virtual_memory()
    gpu_runtime = False
    with h():
        console('pip3 install onnxruntime==1.14.1')
else:
    gpu_runtime = True
    with h():
        console('pip3 install onnxruntime-gpu==1.14.1')
with h():
    deps = [
            'pathvalidate',
            'youtube-dl',
            'django'
    ]
    for dep in deps:
        console('pip3 install {}'.format(dep))
# import modules
#console('pip3 install torch==1.13.1')
console('pip3 install soundfile==0.12.1')
console('pip3 install librosa==0.9.1')
from pathvalidate import sanitize_filename
print('done')
if not gpu_runtime:
    print(f'GPU runtime is disabled. You have {get_size(svmem.total)} RAM.\nProcessing will be incredibly slow. 😈')
elif gpu_info.find('Tesla T4') >= 0:
    print('You got a Tesla T4 GPU. (speeds are around  10-25 it/s)')
elif gpu_info.find('Tesla P4') >= 0:
    print('You got a Tesla P4 GPU. (speeds are around  8-22 it/s)')
elif gpu_info.find('Tesla K80') >= 0:
    print('You got a Tesla K80 GPU. (This is the common gpu, speeds are around 2-10 it/s)')
elif gpu_info.find('Tesla P100') >= 0:
    print('You got a Tesla P100 GPU. (This is the Second to the fastest gpu, speeds are around  15-42 it/s)')
else:
    if gpu_runtime:
        print('You got an unknown GPU. Please report the GPU you got!')
        !nvidia-smi

#console('pip3 install demucs')
#-------------------------------------------------------
# Scripting
mount(mount_to_drive)
toPath('gdrive' if mount_to_drive else 'local')
#check for MDX existence
if not past_installation():
    print('First time installation will take around 3-6 minutes.\nThis requires around 2-3 GB Free Gdrive space.\nPlease try not to interup installation process!!')
    print('Downloading AI...',end=' ')
    with h():
        LoadMDX()
    os.chdir('MDX_Colab')
    print('done')

    print('Downloading models...',end=' ')
    with h():
        #getDemucs('model/')
        getONNX()
    if os.path.isfile('onnx_list'):
        os.remove('onnx_list')
    print('done')

else:
    os.chdir('MDX_Colab')
    update()

################
#outro
print('Success!')

In [ ]:
#@markdown ##Click this to import a ZIP of AUDIO FILES (for isolation.)
#@markdown Or you can use the cell below this to upload files directly instead (which is more convenient) <br> <br>
#@markdown Link the URL path to the audio files (Mega, Drive, etc.) and start the code
url = 'INSERTURLHERE'  #@param {type:"string"}

import subprocess
import os
import shutil
from urllib.parse import urlparse, parse_qs
from google.colab import output
from google.colab import drive


mount_to_drive = True
mount_path = '/content/drive/MyDrive'

def mount(gdrive=False):
    if gdrive:
        if not os.path.exists("/content/drive/MyDrive"):
            try:
                drive.mount("/content/drive", force_remount=True)
            except:
                drive._mount("/content/drive", force_remount=True)
    else:
        pass

mount(mount_to_drive)

def check_package_installed(package_name):
    command = f"pip show {package_name}"
    result = subprocess.run(command.split(), stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    return result.returncode == 0

def install_package(package_name):
    command = f"pip install {package_name} --quiet"
    subprocess.run(command.split(), stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

if not check_package_installed("mega.py"):
    install_package("mega.py")

from mega import Mega
import os
import shutil
from urllib.parse import urlparse, parse_qs
import urllib.parse

!rm -rf /content/unzips/
!rm -rf /content/zips/
!mkdir /content/unzips
!mkdir /content/zips

def sanitize_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            if filename == ".DS_Store" or filename.startswith("._"):
                os.remove(file_path)
        elif os.path.isdir(file_path):
            sanitize_directory(file_path)

audio_zip = urlparse(url).path.split('/')[-2] + '.zip'
audio_zip_path = '/content/zips/' + audio_zip

if url != '':
    if "drive.google.com" in url:
        !gdown $url --fuzzy -O "$audio_zip_path"
    elif "mega.nz" in url:
        m = Mega()
        m.download_url(url, '/content/zips')
    else:
        !wget "$url" -O "$audio_zip_path"

    for filename in os.listdir("/content/zips"):
        if filename.endswith(".zip"):
            zip_file = os.path.join("/content/zips", filename)
            shutil.unpack_archive(zip_file, "/content/unzips", 'zip')

sanitize_directory("/content/unzips")

# Copy the unzipped audio files to the /content/drive/MyDrive/MDX_Colab/tracks folder
!mkdir -p /content/drive/MyDrive/MDX_Colab/tracks
for filename in os.listdir("/content/unzips"):
    if filename.endswith((".wav", ".mp3")):
        audio_file = os.path.join("/content/unzips", filename)
        destination_file = os.path.join("/content/drive/MyDrive/MDX_Colab/tracks", filename)
        shutil.copy2(audio_file, destination_file)
        if os.path.exists(destination_file):
            print(f"Copy successful: {destination_file}")
        else:
            print(f"Copy failed: {audio_file}")

!rm -r /content/unzips/
!rm -r /content/zips/


##Audio Isolation

In [ ]:
#@markdown #Upload your files directly to UVR
#@markdown Run this cell to upload your vocal files that you want to use, (or zip files containing audio), to your Colab. <br>
#@markdown Alternatively, you can upload from the colab files panel, but this should be more convenient. This method may not work on iOS.

from google.colab import files
from IPython.display import display, Javascript
import os
import shutil
import zipfile
import ipywidgets as widgets

# Create the target directory if it doesn't exist
target_dir = '/content/drive/MyDrive/MDX_Colab/tracks'
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

uploaded = files.upload()

for fn in uploaded.keys():
    # Check if the uploaded file is a zip file
    if fn.endswith('.zip'):
        # Write the uploaded zip file to the target directory
        zip_path = os.path.join(target_dir, fn)
        with open(zip_path, 'wb') as f:
            f.write(uploaded[fn])

        unzip_dir = os.path.join(target_dir, fn[:-4])  # Remove the .zip extension from the folder name

        # Extract the zip file
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)

        # Delete the zip file
        if os.path.exists(zip_path):
            os.remove(zip_path)

        print('Zip file "{name}" extracted and removed. Files are in: {folder}'.format(name=fn, folder=unzip_dir))

        # Display copy path buttons for each extracted file
        for extracted_file in os.listdir(unzip_dir):
            extracted_file_path = os.path.join(unzip_dir, extracted_file)
            extracted_file_length = os.path.getsize(extracted_file_path)

            extracted_file_label = widgets.HTML(
                value='Extracted file "{name}" with length {length} bytes'.format(name=extracted_file, length=extracted_file_length)
            )
            display(extracted_file_label)

            extracted_file_path_text = widgets.HTML(
                value='File saved to: <a href="{}" target="_blank">{}</a>'.format(extracted_file_path, extracted_file_path)
            )

            extracted_copy_button = widgets.Button(description='Copy')
            extracted_copy_button_file_path = extracted_file_path  # Make a local copy of the file path

            def copy_to_clipboard(b):
                js_code = '''
                    const el = document.createElement('textarea');
                    el.value = "{path}";
                    el.setAttribute('readonly', '');
                    el.style.position = 'absolute';
                    el.style.left = '-9999px';
                    document.body.appendChild(el);
                    el.select();
                    document.execCommand('copy');
                    document.body.removeChild(el);
                '''
                display(Javascript(js_code.format(path=extracted_copy_button_file_path)))

            extracted_copy_button.on_click(copy_to_clipboard)
            display(widgets.HBox([extracted_file_path_text, extracted_copy_button]))

        continue

    # For non-zip files
    # Save the file to the target directory
    file_path = os.path.join(target_dir, fn)
    with open(file_path, 'wb') as f:
        f.write(uploaded[fn])

    file_length = len(uploaded[fn])
    file_label = widgets.HTML(
        value='User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=file_length)
    )
    display(file_label)

    # Check if the uploaded file is a .pth or .index file
    if fn.endswith('.pth') or fn.endswith('.index'):
        warning_text = widgets.HTML(
            value='<b style="color: red;">Warning:</b> You are uploading a model file in the wrong place. Please ensure it is uploaded to the correct location.'
        )
        display(warning_text)

    # Create a clickable path with copy button
    file_path_text = widgets.HTML(
        value='File saved to: <a href="{}" target="_blank">{}</a>'.format(file_path, file_path)
    )

    copy_button = widgets.Button(description='Copy')
    copy_button_file_path = file_path  # Make a local copy of the file path

    def copy_to_clipboard(b):
        js_code = '''
            const el = document.createElement('textarea');
            el.value = "{path}";
            el.setAttribute('readonly', '');
            el.style.position = 'absolute';
            el.style.left = '-9999px';
            document.body.appendChild(el);
            el.select();
            document.execCommand('copy');
            document.body.removeChild(el);
        '''
        display(Javascript(js_code.format(path=copy_button_file_path)))

    copy_button.on_click(copy_to_clipboard)
    display(widgets.HBox([file_path_text, copy_button]))

# Remove the original uploaded files from /content/
for fn in uploaded.keys():
    if os.path.exists(os.path.join("/content/", fn)):
        os.remove(os.path.join("/content/", fn))

In [ ]:
#@markdown ### Print a list of tracks
for i in glob.glob('tracks/*'):
    print(os.path.basename(i))

In [ ]:
if not 'initialised' in globals():
    raise NameError('Please run the first cell first!! #scrollTo=H_cTbwhVq4K6')

#import all models metadata
import json
with open('model_data.json', 'r') as f:
  model_data = json.load(f)

# Modifiable variables
tracks_path = 'tracks/'
separated_path = 'separated/'

#@markdown ### Input track
#@markdown Enter any link/Filename (Upload your songs in tracks folder)
track = "Butterfly.wav" #@param {type:"string"}

#@markdown ---
#@markdown ### Models
ONNX = "MDX-UVR Ins Model Full Band 498 (HQ_2)" #@param ["off", "Karokee", "Karokee_AGGR", "Karokee 2", "baseline", "MDX-UVR Ins Model 415", "MDX-UVR Ins Model 418", "MDX-UVR Ins Model 464", "MDX-UVR Ins Model 496 - inst main-MDX 2.1", "Kim ft other instrumental model", "MDX-UVR Vocal Model 427", "MDX-UVR-Kim Vocal Model (old)", "MDX-UVR Ins Model Full Band 292", "MDX-UVR Ins Model Full Band 403", "MDX-UVR Ins Model Full Band 450 (HQ_1)", "MDX-UVR Ins Model Full Band 498 (HQ_2)"]
Demucs = 'off'#@param ["off","demucs_extra"]

#@markdown ---
#@markdown ### Parameters
denoise = False #@param {type:"boolean"}
normalise = True #@param {type:"boolean"}
#getting values from model_data.json related to ONNX var (model folder name)
amplitude_compensation = model_data[ONNX]["compensate"]
dim_f = model_data[ONNX]["mdx_dim_f_set"]
dim_t = model_data[ONNX]["mdx_dim_t_set"]
n_fft = model_data[ONNX]["mdx_n_fft_scale_set"]

mixing_algorithm = 'max_mag' #@param ["default","min_mag","max_mag"]
chunks = 55 #@param {type:"slider", min:1, max:55, step:1}
shifts = 10 #@param {type:"slider", min:0, max:10, step:0.1}

##validate values
track = track if 'http' in track else tracks_path+track
normalise = '--normalise' if normalise else ''
denoise = '--denoise' if denoise else ''

if ONNX == 'off':
    pass
else:
    ONNX = 'onnx/'+ONNX
if Demucs == 'off':
    pass
else:
    Demucs = 'model/'+Demucs+'.th'
#@markdown ---
#@markdown ### Stems
bass = False #@param {type:"boolean"}
drums = False #@param {type:"boolean"}
others = False #@param {type:"boolean"}
vocals = True #@param {type:"boolean"}
#@markdown ---
#@markdown ### Invert stems to mixture
invert_bass = False #@param {type:"boolean"}
invert_drums = False #@param {type:"boolean"}
invert_others = False #@param {type:"boolean"}
invert_vocals = True #@param {type:"boolean"}
invert_stems = []
stems = []
if bass:
    stems.append('b')
if drums:
    stems.append('d')
if others:
    stems.append('o')
if vocals:
    stems.append('v')

invert_stems = []
if invert_bass:
    invert_stems.append('b')
if invert_drums:
    invert_stems.append('d')
if invert_others:
    invert_stems.append('o')
if invert_vocals:
    invert_stems.append('v')

margin = 44100

###
# incompatibilities
###

console(f"python main.py --n_fft {n_fft} --dim_f {dim_f} --dim_t {dim_t} --margin {margin} -i \"{track}\" --mixing {mixing_algorithm} --onnx \"{ONNX}\" --model {Demucs} --shifts {round(shifts)} --stems {''.join(stems)} --invert {''.join(invert_stems)} --chunks {chunks} --compensate {amplitude_compensation} {normalise} {denoise}")

<sup>Models provided are from [Kuielab](https://github.com/kuielab/mdx-net-submission/), [UVR](https://github.com/Anjok07/ultimatevocalremovergui/) and [Kim](https://github.com/KimberleyJensen/) <br> (you can support UVR [here](https://www.buymeacoffee.com/uvr5/vip-model-download-instructions) and [here](https://boosty.to/uvr)).</sup></br>
<sup>Original UVR notebook by [Audio Hacker](https://www.youtube.com/channel/UC0NiSV1jLMH-9E09wiDVFYw/), modified by Audio Separation community & then kalomaze (for RVC colab).</sup></br>
<sup>Big thanks to the [Audio Separation Discord](https://discord.gg/zeYU2Wzbgj) for helping me implement this in the colab.</sup></br>

##**UVR Colab Settings explanation**<br>

The defaults already provided are generally recommended. However, if you would like to try tweaking them, here's an explanation:

*Mixing algorithm* - max_mag - is generally for vocals (gives the most residues in instrumentals), min_mag - for instrumentals (the most aggresive) though "min_mag solve some un-wanted vocal soundings, but instrumental [is] more muffled and less detailed". Check out also "default" as it's in between both - a.k.a. average (it's also required for Demucs enabled which works only for vocal models).<br>

*Chunks* - Set it to 55 or 40 (less aggressive) to alleviate some occasional instrument dissapearing.
Set 1 for the best clarity. It works for at least instrumental model (4:15 track, at least for Tesla T4 (shown at the top) generally better quality, but some instruments tend to disappear more using 1 than 10. For Demucs enabled and/or vocal model it can be set to 10 if your track is below 5:00 minutes. The more chunks, the faster separation up to ~40. For 4:15 track, 72 is max supported till memory allocation error shows up (disabled chunks returns error too). <br>

*Shifts* - can be set max to 10, but it only slightly increases SDR, while processing time is 1.7x longer for each shift and it gives similar result to shifts 5.

*Normalization* - "normalizes all input at first and then changes the wave peak back to original. This makes the separation process better, also less noise" (e.g. if you have to noisy hihats or big amplitude compensation - disable it).
<br>

*Demucs* enabled works correctly with mixing algorithm set to default and only with vocal models (Kim and 427). It's also the only option to get rid of noise of MDX models. Normalization enabled is necessary (but that cnfiguration has slightly more vocal residues than instrumental model). Decrease chunks to 40 if you have ONNXRuntimeError with Demucs on (it requires lower chunks).
<br>

##**Recommended models**<br>

For vocals (by raw SDR output, not factoring in manual cleanup):
- Kim vocal 2 (less instrumental residues in vocal stem)
- Kim vocal 1
<br>or alternatively
- 427
- 406

For best lead vocals:
- Karaokee 2

For best backing vocals:
- [HP_KAROKEE-MSB2-3BAND-3090](https://colab.research.google.com/drive/16Q44VBJiIrXOgTINztVDVeb0XKhLKHwl?usp=sharing)

It's rather inconvenient that the VR Architecture models aren't here and have to be run through the above colab, but they can't coexist in the same colab as of right now. I will attempting a better solution in the future.